In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors # unsupervise learning

pd.options.display.max_columns = 100

In [2]:
prod = pd.read_csv("product.csv")
df = pd.read_csv("201904 sales reciepts.csv")
df.head(10)

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N
5,39,2019-04-01,18:54:46,3,17,664,Y,1,1,32,1,3.00,3.00,N
6,50,2019-04-01,13:03:49,3,12,316,N,1,1,49,2,6.00,3.00,N
7,53,2019-04-01,11:21:14,3,12,38,N,1,1,60,1,3.75,3.75,N
8,59,2019-04-01,19:30:55,3,12,370,Y,1,1,51,2,6.00,3.00,N
9,62,2019-04-01,12:01:00,3,12,180,Y,1,1,49,2,6.00,3.00,N


In [3]:
prod

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,Add-ons,Flavours,Regular syrup,Chocolate syrup,Bursting with chocolate flavour,pump,0.04,$0.80,Y,N,N
84,85,Beverages,Coffee,Specialty coffee,Rio Nights,2 shots of Ouro Brasilerio and pure cane sugar...,6 oz,1.20,$6.00,Y,N,Y
85,87,Beverages,Coffee,Barista Espresso,Ouro Brasileiro shot,From Rio,1.5 oz,0.60,$3.00,Y,N,N
86,88,Beverages,Coffee,Barista Espresso,Ouro Brasileiro shot promo,Ouro promo,16 oz,0.42,$2.10,Y,Y,N


In [7]:
# df = pd.merge(df, prod, on='product_id')
df.head(10)

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.5,2.5,N,Beverages,Tea,Brewed Chai tea,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,0.63,$2.50,Y,N,N
1,249,2019-04-01,11:15:38,3,12,22,Y,1,1,52,1,2.5,2.5,N,Beverages,Tea,Brewed Chai tea,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,0.63,$2.50,Y,N,N
2,326,2019-04-01,12:05:31,3,12,659,N,1,1,52,2,5.0,2.5,N,Beverages,Tea,Brewed Chai tea,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,0.63,$2.50,Y,N,N
3,497,2019-04-01,16:49:38,3,12,407,Y,1,1,52,1,2.5,2.5,N,Beverages,Tea,Brewed Chai tea,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,0.63,$2.50,Y,N,N
4,541,2019-04-01,19:17:34,3,12,537,Y,1,1,52,2,5.0,2.5,N,Beverages,Tea,Brewed Chai tea,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,0.63,$2.50,Y,N,N
5,585,2019-04-01,17:58:05,3,17,385,N,1,1,52,2,5.0,2.5,N,Beverages,Tea,Brewed Chai tea,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,0.63,$2.50,Y,N,N
6,828,2019-04-01,11:14:07,3,12,456,Y,1,1,52,1,2.5,2.5,N,Beverages,Tea,Brewed Chai tea,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,0.63,$2.50,Y,N,N
7,1015,2019-04-01,11:40:26,3,12,7,N,1,1,52,2,5.0,2.5,N,Beverages,Tea,Brewed Chai tea,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,0.63,$2.50,Y,N,N
8,1386,2019-04-01,15:36:03,3,17,441,N,1,1,52,1,2.5,2.5,N,Beverages,Tea,Brewed Chai tea,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,0.63,$2.50,Y,N,N
9,1389,2019-04-01,11:14:45,3,17,212,Y,1,1,52,2,5.0,2.5,N,Beverages,Tea,Brewed Chai tea,Traditional Blend Chai Rg,Sit back and think of the tropical breezes.,16 oz,0.63,$2.50,Y,N,N


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49894 entries, 0 to 49893
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   transaction_id           49894 non-null  int64  
 1   transaction_date         49894 non-null  object 
 2   transaction_time         49894 non-null  object 
 3   sales_outlet_id          49894 non-null  int64  
 4   staff_id                 49894 non-null  int64  
 5   customer_id              49894 non-null  int64  
 6   instore_yn               49894 non-null  object 
 7   order                    49894 non-null  int64  
 8   line_item_id             49894 non-null  int64  
 9   product_id               49894 non-null  int64  
 10  quantity                 49894 non-null  int64  
 11  line_item_amount         49894 non-null  float64
 12  unit_price               49894 non-null  float64
 13  promo_item_yn            49894 non-null  object 
 14  product_group         

In [18]:
# df.groupby()

cust = df.groupby(['customer_id','product'])['product'].count().to_frame()
cust

product
customer_id product                              
0           Almond Croissant                  324
            Brazilian - Organic                42
            Brazilian Lg                      452
            Brazilian Rg                      478
            Brazilian Sm                      442
...                                           ...
8501        Earl Grey Lg                        1
            Lemon Grass Lg                      1
            Peppermint Lg                       2
            Serenity Green Tea Lg               1
            Sustainably Grown Organic Lg        1

[22026 rows x 1 columns]

In [29]:
product_match = pd.crosstab(index=df['customer_id'],
            columns=df['product'],
            values=df['product'],aggfunc=pd.Series.nunique
           ).fillna(0)

product_match

product,Almond Croissant,Brazilian - Organic,Brazilian Lg,Brazilian Rg,Brazilian Sm,Cappuccino,Cappuccino Lg,Carmel syrup,Chili Mayan,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Civet Cat,Columbian Medium Roast,Columbian Medium Roast Lg,Columbian Medium Roast Rg,Columbian Medium Roast Sm,Cranberry Scone,Croissant,Dark chocolate,Dark chocolate Lg,Dark chocolate Rg,Earl Grey,Earl Grey Lg,Earl Grey Rg,English Breakfast,English Breakfast Lg,English Breakfast Rg,Espresso Roast,Espresso shot,Ethiopia,Ethiopia Lg,Ethiopia Rg,Ethiopia Sm,Ginger Biscotti,Ginger Scone,Guatemalan Sustainably Grown,Hazelnut Biscotti,Hazelnut syrup,I Need My Bean! Diner mug,I Need My Bean! Latte cup,I Need My Bean! T-shirt,Jamacian Coffee River,Jamaican Coffee River Lg,Jamaican Coffee River Rg,Jamaican Coffee River Sm,Jumbo Savory Scone,Latte,Latte Rg,Lemon Grass,Lemon Grass Lg,Lemon Grass Rg,Morning Sunrise Chai,Morning Sunrise Chai Lg,Morning Sunrise Chai Rg,Oatmeal Scone,Organic Decaf Blend,Our Old Time Diner Blend,Our Old Time Diner Blend Lg,Our Old Time Diner Blend Rg,Our Old Time Diner Blend Sm,Ouro Brasileiro shot,Peppermint,Peppermint Lg,Peppermint Rg,Primo Espresso Roast,Scottish Cream Scone,Serenity Green Tea,Serenity Green Tea Lg,Serenity Green Tea Rg,Spicy Eye Opener Chai,Spicy Eye Opener Chai Lg,Spicy Eye Opener Chai Rg,Sugar Free Vanilla syrup,Sustainably Grown Organic,Sustainably Grown Organic Lg,Sustainably Grown Organic Rg,Traditional Blend Chai,Traditional Blend Chai Lg,Traditional Blend Chai Rg
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8497,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0

In [31]:
liked_product ='Cappuccino'
liked_product_selling = product_match[liked_product]
liked_product_selling

customer_id
0       1.0
1       0.0
2       0.0
3       0.0
4       1.0
       ... 
8497    0.0
8498    0.0
8499    0.0
8500    0.0
8501    0.0
Name: Cappuccino, Length: 2248, dtype: float64

In [32]:
similar_liked_product = product_match.corrwith(liked_product_selling)
similar_liked_product

product
Almond Croissant                0.069728
Brazilian - Organic             0.007370
Brazilian Lg                    0.021943
Brazilian Rg                    0.010277
Brazilian Sm                   -0.001251
                                  ...   
Sustainably Grown Organic Lg   -0.003021
Sustainably Grown Organic Rg    0.018255
Traditional Blend Chai         -0.015120
Traditional Blend Chai Lg      -0.008506
Traditional Blend Chai Rg      -0.003373
Length: 80, dtype: float64

In [33]:
df_product_corr = pd.DataFrame(similar_liked_product, columns=['Pearson_Corr'])
df_product_corr.dropna(inplace=True)
df_product_corr.sort_values('Pearson_Corr', ascending=False).head(10)

,Pearson_Corr
product,
Cappuccino,1.000000
Chocolate syrup,0.165163
Sugar Free Vanilla syrup,0.120743
Carmel syrup,0.102853
Hazelnut syrup,0.082475
Ginger Biscotti,0.078313
Almond Croissant,0.069728
Chocolate Chip Biscotti,0.066077
Hazelnut Biscotti,0.054000


In [34]:
movie_matrix = csr_matrix(product_match.values)
movie_matrix

<2248x80 sparse matrix of type '<class 'numpy.float64'>'
	with 22026 stored elements in Compressed Sparse Row format>

In [35]:
print(movie_matrix[:10])

  (0, 0)	1.0
  (0, 1)	1.0
  (0, 2)	1.0
  (0, 3)	1.0
  (0, 4)	1.0
  (0, 5)	1.0
  (0, 6)	1.0
  (0, 7)	1.0
  (0, 8)	1.0
  (0, 9)	1.0
  (0, 10)	1.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 13)	1.0
  (0, 14)	1.0
  (0, 15)	1.0
  (0, 16)	1.0
  (0, 17)	1.0
  (0, 18)	1.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 21)	1.0
  (0, 22)	1.0
  (0, 23)	1.0
  (0, 24)	1.0
  :	:
  (7, 72)	1.0
  (7, 79)	1.0
  (8, 9)	1.0
  (8, 16)	1.0
  (8, 18)	1.0
  (8, 27)	1.0
  (8, 31)	1.0
  (8, 32)	1.0
  (8, 44)	1.0
  (8, 47)	1.0
  (8, 48)	1.0
  (8, 51)	1.0
  (8, 64)	1.0
  (8, 79)	1.0
  (9, 6)	1.0
  (9, 11)	1.0
  (9, 16)	1.0
  (9, 27)	1.0
  (9, 31)	1.0
  (9, 34)	1.0
  (9, 44)	1.0
  (9, 52)	1.0
  (9, 53)	1.0
  (9, 64)	1.0
  (9, 72)	1.0


In [36]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(movie_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [38]:
film = 'Cappuccino'
item_id = int(pd.DataFrame(df['product_id'][df['product']==film]).iloc[0])

distance, idx = model.kneighbors(product_match.loc[item_id, :].values.reshape(1,-1), n_neighbors=6)
distance, idx

#402 itu si batman, karena distance nya 0, dia menghitung jarak dengan dirinya sendiri

(array([[0.        , 0.39697731, 0.43019712, 0.45454545, 0.46064011,
         0.46699821]]),
 array([[  40, 1052, 1302, 1777,  362,  667]]))

In [41]:
num = 1
for j, i in zip(distance[0], idx[0]):
    if j == 0:
        print(f"Rekomendasi untuk film {df[df['product_id'] == item_id]['product'].iloc[0]}")
    else :
        print(f"{num}: Film berjudul {df[df['product_id'] == i]['product'].iloc[0]} dengan distance {j}")
        num +=1

Rekomendasi untuk film Cappuccino


IndexError: single positional indexer is out-of-bounds